## Observability with MLflow and LangGraph

### Overview

This demo introduces **MLflow observability** into LangGraph workflows. By tracing and logging each step of a workflow—including LLM invocations and tool usage—developers can monitor, debug, and analyze their pipelines directly from the MLflow UI.

In [21]:
import mlflow
import os
from typing import Dict
from tavily import TavilyClient
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import MessagesState
from langgraph.graph import START, END, StateGraph
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage, SystemMessage
from IPython.display import Image, display

### **MLflow Setup**

#### a. **Tracking Configuration**

- Start a local MLflow server in terminal: `mlflow server`
- A local MLflow server is assumed to be running at `http://127.0.0.1:5000`.
- This address is set as the MLflow tracking URI.

In [22]:
tracking_uri = "http://127.0.0.1:5000"
mlflow.set_tracking_uri(tracking_uri)

In [23]:
experiment = mlflow.set_experiment("demo")

#### b. **Manual Trace Example**

- A simple `add()` function is traced with `@mlflow.trace`.
- Inputs and outputs are automatically logged to the MLflow UI.

In [24]:
@mlflow.trace
def add(a, b):
    return a + b

In [25]:
add(1, 2)

3

Trace(request_id=7160871f70294f1799d8f5f02d9988a8)

### **Workflow Overview**

- The workflow uses a similar structure to previous demos:
    - A question is passed to an **entry_point**.
    - The **agent** determines whether a **tool** (web search) is needed.
    - If required, the **tool** is called and results are returned.
- The workflow interrupts at a **breakpoint before tools**, enabling streaming inspection.

In [26]:
from dotenv import load_dotenv
load_dotenv()
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0,
)

In [27]:
@tool
def web_search(question:str)->Dict:
    """
    Return top search results for a given search query
    """
    tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
    response = tavily_client.search(question)
    return response

In [28]:
class State(MessagesState):
    question: str
    answer: str

In [29]:
llm_with_tools = llm.bind_tools([web_search])

In [30]:
def entry_point(state: State):
    question = state["question"]
    system_message = SystemMessage("You conduct web search to respond to user's questions")
    human_message = HumanMessage(question)
    messages = [system_message, human_message]
    return {"messages": messages}

In [31]:
def agent(state: State):
    messages = state["messages"]
    ai_message = llm_with_tools.invoke(messages)
    return {"messages": ai_message, "answer": ai_message.content}

In [32]:
def router(state: MessagesState):
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        return "tools"

    return END

In [33]:
workflow = StateGraph(State)
workflow.add_node("entry_point", entry_point)
workflow.add_node("agent", agent)
workflow.add_node("tools", ToolNode([web_search]))

workflow.add_edge(START, "entry_point")
workflow.add_edge("entry_point", "agent")
workflow.add_conditional_edges(
    source="agent", 
    path=router, 
    path_map=["tools", END]
)
workflow.add_edge("tools", "agent")

In [34]:
memory = MemorySaver()
graph = workflow.compile(
    interrupt_before=["tools"], 
    checkpointer=memory
)

In [36]:
# display(
#     Image(
#         graph.get_graph().draw_mermaid_png()
#     )
# )

### **LangChain Autologging Integration**

- `mlflow.langchain.autolog()` is called to enable automatic logging of LangChain events:
    - LLM inputs and outputs
    - Tool call traces
    - Message sequences
    - Token usage and performance metrics

In [37]:
mlflow.langchain.autolog()

### **Execution Example**

- Input question: *"What is the capital of Brazil?"*
- Initial invocation:
    - System and human messages are appended.
    - The agent node recognizes the need for a tool call (no direct answer yet).
- MLflow logs the trace up to the **tool call breakpoint**.

In [38]:
input_question = {"question": "what's the capital of Brazil?"}
config = {"configurable": {"thread_id": 1}}

In [39]:
for event in graph.stream(input=input_question, config=config, stream_mode="values"):
    if not event['messages']:
        continue
    event['messages'][-1].pretty_print()

================================ Human Message =================================

what's the capital of Brazil?


INFO:httpx:HTTP Request: POST https://openai.vocareum.com/v1/chat/completions "HTTP/1.1 200 OK"


================================== Ai Message ==================================
Tool Calls:
  web_search (call_wbigPKFc7HYP8LhLIPsd8Dc1)
 Call ID: call_wbigPKFc7HYP8LhLIPsd8Dc1
  Args:
    question: What is the capital of Brazil?


Trace(request_id=ac6d4b3775f846be9da318ac410c8351)

#### a. **Tool Node Execution**

- The tool node (Tavily web search) is executed.
- Output: Top results related to the question are logged.

In [40]:
state = graph.get_state(config=config)

In [41]:
state.next

('tools',)

In [42]:
for event in graph.stream(input=None, config=config, stream_mode="values"):
    if not event['messages']:
        continue
    event['messages'][-1].pretty_print()

================================== Ai Message ==================================
Tool Calls:
  web_search (call_wbigPKFc7HYP8LhLIPsd8Dc1)
 Call ID: call_wbigPKFc7HYP8LhLIPsd8Dc1
  Args:
    question: What is the capital of Brazil?
================================= Tool Message =================================
Name: web_search

{"query": "What is the capital of Brazil?", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Brasília - Wikipedia", "url": "https://en.wikipedia.org/wiki/Brasília", "content": "445 Region Latin America and the Caribbean Brasília (/brəˈzɪliə/ brə-ZIL-ee-ə,[4][5] Portuguese: [bɾaˈzili.ɐ, bɾaˈziljɐ] ⓘ) is the capital city of Brazil. Located in the Brazilian highlands in the country's Central-West region, it is the seat of government of the Federal District. It was founded by President Juscelino Kubitschek on 21 April 1960, to replace Rio de Janeiro as the national capital. [6] Brasília is a planned city developed by Lúcio Costa, Osc

INFO:httpx:HTTP Request: POST https://openai.vocareum.com/v1/chat/completions "HTTP/1.1 200 OK"


================================== Ai Message ==================================

The capital of Brazil is Brasília. It was founded on April 21, 1960, to replace Rio de Janeiro as the national capital. Brasília is located in the Central-West region of Brazil and is known for its modernist architecture and urban planning. 

For more information, you can visit the [Wikipedia page on Brasília](https://en.wikipedia.org/wiki/Brasília).


Trace(request_id=bc4048e9f9a54a95854f3cb9ea7d613d)

### **Reviewing the Trace in MLflow UI**

- Each node’s inputs and outputs are logged:
    - Entry point: initial user input and message formatting.
    - Agent: LLM messages and tool call info.
    - Tool: external API invocation and response data.
    - Final agent call: formatted answer to the user.
- MLflow panels show:
    - Run timeline
    - Artifact logs
    - Token counts
    - Inputs/outputs per node

### **Conclusion**

- **Breakpoints** provide pause-and-inspect control.
- **Traces** log the full context of decision-making and tool use.
- **MLflow + LangChain** combination brings transparency and observability to LLM-based workflows.
- Easy to debug issues, understand performance, and trace final outputs to original prompts or tool responses.

- Integrating **MLflow with LangGraph and LangChain** gives developers critical insight into how AI workflows behave.
- Each component’s behavior becomes traceable, auditable, and optimizable.
- This observability is crucial for safe, production-grade LLM applications.